# 제주비건지도 크롤링

In [2]:
from bs4 import BeautifulSoup

from selenium import webdriver

import re

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd

In [4]:
base_url = "http://www.jejuvegan.com/vegan_map"

In [5]:
driver = webdriver.Chrome(executable_path = 'D:\Python_417\chromedriver')
driver.get(url=base_url)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

In [69]:
raw = soup.select('div.col-md-9.wz_title')

In [377]:
final_df = pd.DataFrame(columns=['title', 'option', 'phone', 'menu', 'work', 'off', 'address'])
final_df['sns'], final_df['etc'] = 'empty', 'empty'
final_df

,title,option,phone,menu,work,off,address,sns,etc


In [378]:
def option_function (option):
    option = option[1:-2]
    if option == '/img/vegan_icon_s.png':
        option = '비건 식당'
    elif option == '/img/vegan_option_icon_s.png':
        option = '비건 옵션'
    elif option == '/img/dessert_s_icon.png':
        option = '비건 디저트'
    else:
        option = '베이커리'
    return option

In [379]:
gettext_function = lambda x : x.get_text().strip()

In [382]:
def info_function(content):
    basic_idx = [2,7,10,13]
    phone, sns, etc = 'empty', 'empty', 'empty'
    if len(content)<=16:
        if '연락처: ' == content[1].get_text():
            phone, menu, work, off = [gettext_function(content[x]) for x in basic_idx]
            address = gettext_function(content[15])
        else:
            menu, work, off, address = [gettext_function(content[x]) for x in [2,5,7,10]]
    elif len(content)<=21:
        phone, menu, work, off = [gettext_function(content[x]) for x in basic_idx]
        address = gettext_function(content[15])
        sns = content[18]['href']
    else: # len(content) >=21:
        phone, menu, work, off = [gettext_function(content[x]) for x in basic_idx]
        address, etc =  [gettext_function(content[x]) for x in [15, 18]]
        sns = content[20]['href']
    return phone, menu, work, off, address, sns, etc

In [383]:
 def vegan_crawling(line):
    # 식당 이름
    title = line.select('div > h3 > a')[0].get_text()
    # 식당 옵션 (비건 식당, 비건 옵션, 비건 디저트, 비건 베이커리)
    option = str(line.select('div > img')[0])
    option = re.search('".*" ', option).group()
    option = option_function(option)
    content = line.p.contents
    try:
        phone, menu, work, off, address, sns, etc = info_function(content)
    except:
        phone, menu, work, off, address, sns, etc = ['error'] * 7
    return {'title' : title, 'option' : option, 'phone' : phone,
            'menu' : menu, 'work' : work, 'off' : off, 'address' : address,
            'etc' : etc, 'sns' : sns}

In [384]:
for idx, line in enumerate(raw):
    print(idx)
    final_df = final_df.append(vegan_crawling(line), ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [385]:
final_df['phone'].value_counts() # error 3개 정도는 손으로 바꿔줌

empty             9
error             3
010-2700-0999     1
0507-1445-3444    1
0507-1409-1407    1
                 ..
010-5419-8789     1
064-773-1377      1
0507-1392-3444    1
064-796-9871      1
0507-1385-4711    1
Name: phone, Length: 88, dtype: int64

In [386]:
final_df.to_csv('././bigcon_data/제주비건식당크롤링.csv', encoding='utf-8-sig', index=False)